In [ ]:
import pandas as pd
import csv
from math import isnan
import numpy as np
from datetime import date
import pickle
import time
dateparse = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d') if str(x)!="nan" else "0"

chunkcount=0

def avg(l):
	return sum(l)/len(l)

print("starting to read train")
chunksize = 10 ** 6
train = pd.read_csv("../input/expedia-hotel-recommendations/train.csv",chunksize=chunksize,
    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32,
                             'srch_adults_cnt':np.int32,"srch_children_cnt":np.int32,"srch_rm_cnt":np.int32,
                             "orig_destination_distance":np.float32,
          'user_location_country':np.int32,"user_location_region":np.int32,"user_location_city":np.int32,"srch_destination_type_id":np.int32,"hotel_continent":np.int32,"hotel_country":np.int32,"hotel_market":np.int32,"srch_adults_cnt":np.int32,"srch_children_cnt":np.int32,"srch_rm_cnt":np.int32,"srch_destination_id":np.int32},
    parse_dates=['srch_ci','srch_co'], date_parser=dateparse,
	usecols=['orig_destination_distance',"is_booking","hotel_cluster",'user_location_country',"user_location_region","user_location_city","srch_destination_type_id","hotel_continent","hotel_country","hotel_market","srch_adults_cnt","srch_children_cnt","srch_rm_cnt","srch_ci","srch_co","user_id","srch_destination_id"])
print("read train.")
maxuid=0
max_stay= 370
max_user_id= 9999
max_user_location_country= 239
max_user_location_region =  1017
max_user_location_city =  56507
max_srch_destination_type_id= 9
max_srch_destination_id= 65068
max_hotel_continent= 6
max_hotel_country= 212
max_hotel_market= 2117
max_srch_adults_cnt= 9
max_srch_children_cnt= 9
max_srch_rm_cnt= 8
min_stay= 0
min_user_id= 0
min_user_location_country= 0
min_user_location_region =  0
min_user_location_city =  0
min_srch_destination_type_id= 1
min_srch_destination_id= 4
min_hotel_continent= 0
min_hotel_country= 0
min_hotel_market= 0
min_srch_adults_cnt= 0
min_srch_children_cnt= 0
min_srch_rm_cnt= 1

ucount=0
max_min_user_location_country = max_user_location_country - min_user_location_country
max_min_user_location_region = max_user_location_region - min_user_location_region
max_min_user_location_city = max_user_location_city - min_user_location_city
max_min_srch_destination_type_id = max_srch_destination_type_id - min_srch_destination_type_id
max_min_srch_destination_id = max_srch_destination_id - min_srch_destination_id
max_min_hotel_continent = max_hotel_continent - min_hotel_continent
max_min_hotel_country = max_hotel_country - min_hotel_country
max_min_hotel_market = max_hotel_market - min_hotel_market
max_min_srch_adults_cnt = max_srch_adults_cnt - min_srch_adults_cnt
max_min_srch_children_cnt = max_srch_children_cnt - min_srch_children_cnt
max_min_srch_rm_cnt = max_srch_rm_cnt - min_srch_rm_cnt
max_min_stay = max_stay - min_stay
max_min_user_id = max_user_id - min_user_id
def abs(x):
    if x<0:
        return -x
    else:
        return x
vectors = []
outputs = []
for chunk in train:
    chunkcount+=1
    print("chunk",chunkcount,chunk.shape)
    if chunkcount<=26:
        continue
    starttime = time.time()
    for ind in chunk.index:
        ci = chunk["srch_ci"][ind]
        co = chunk["srch_co"][ind]
        if ci=="0" or co=="0":
            continue
        uid = chunk['user_id'][ind]
        odd = chunk['orig_destination_distance'][ind]
        if isnan(odd):
            continue
        date_diff = abs(pd.Timedelta(co - ci).days)
        ucount+=1
        user_location_countries = chunk["user_location_country"][ind]
        user_location_regions = chunk["user_location_region"][ind]
        user_location_cities = chunk["user_location_city"][ind]
        srch_destination_type_ids = chunk["srch_destination_type_id"][ind]
        hotel_continents = chunk["hotel_continent"][ind]
        hotel_countries = chunk["hotel_country"][ind]
        hotel_markets = chunk["hotel_market"][ind]
        srch_adults_cnts = chunk["srch_adults_cnt"][ind]
        srch_children_cnts = chunk["srch_children_cnt"][ind]
        srch_rm_cnts = chunk["srch_rm_cnt"][ind]
        stay = date_diff
        user_ids = chunk["user_id"][ind]
        srch_destination_ids = chunk["srch_destination_id"][ind]
        d1 =  ci
        did = chunk['srch_destination_id'][ind]
        isweekend = 0
        if d1.weekday() > 3:
            isweekend = 1
        ismonthend = 0
        if abs(d1.day - 30)< 3 or abs(d1.day - 1)<3:
            ismonthend = 1
        is_summer=0
        is_winter=0
        is_spring=0
        is_rainy=0
        mo = d1.month
        da = d1.day
        if mo>1 and mo<4:
            is_spring = 1
        elif mo<7:
            is_summer = 1
        elif mo<10:
            is_rainy = 1
        else:
            is_winter = 1
        adults = chunk['srch_adults_cnt'][ind]
        children = chunk['srch_children_cnt'][ind]
        rm = chunk['srch_rm_cnt'][ind]
        is_family = 0
        if adults>0 and children>0:
            is_family = 1
        is_trip = 0
        if rm>5:
            is_trip = 1
        is_valentines = 0
        is_christmas = 0
        if mo==2 and abs(da-14)<2:
            is_valentines = 1
        if mo==12 and abs(da-25)<5:
            is_christmas = 1
        vector = [odd, user_location_countries,user_location_regions,user_location_cities,srch_destination_type_ids,hotel_continents, hotel_countries,hotel_markets,srch_adults_cnts,srch_children_cnts,srch_rm_cnts,stay,user_ids,srch_destination_ids,isweekend,ismonthend, is_valentines,is_christmas,
           is_spring,is_summer,is_rainy,is_winter,is_family,is_trip]
        output = [chunk["is_booking"][ind],chunk["hotel_cluster"][ind]]
        vectors.append(vector)
        outputs.append(output)
        vector=[]
        output=[]
    print("processed",chunkcount,len(outputs),time.time()-starttime,"seconds")
    with open("vectors"+str(chunkcount)+".data","wb") as file:
        pickle.dump(vectors,file)
    with open("outputs"+str(chunkcount)+".data","wb") as file:
        pickle.dump(outputs,file)
    vectors = []
    outputs = []
# print(nans)



In [ ]:
import pickle
from sklearn.model_selection import train_test_split
X_train = []
y_train = []
X_test = []
y_test = []
X_valid = []
y_valid = []
path = "vectors1.data"
print(path)
file1 = open(path,"rb")
path = "outputs1.data"
print(path)
file2 = open(path,"rb")
featurevects = pickle.load(file1)
op = pickle.load(file2)
outputs = []
for i in op:
    outputs.append(i[1])
X = featurevects[:100000]
y = outputs[:100000]
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.3)
X_tr, X_valid, y_tr, y_valid = train_test_split(X_train,y_train, stratify=y_train, test_size=0.3)

with open("xgbtrain_vect","wb") as file:
    pickle.dump(X_tr,file)
with open("xgbtrain_op","wb") as file:
    pickle.dump(y_tr,file)
with open("xgbvalid_vect","wb") as file:
    pickle.dump(X_valid,file)
with open("xgbvalid_op","wb") as file:
    pickle.dump(y_valid,file)
with open("X_test","wb") as file:
    pickle.dump(X_test,file)
with open("y_test","wb") as file:
    pickle.dump(y_test,file)
with open("rftrain_vect","wb") as file:
    pickle.dump(X_train,file)
with open("rftrain_op","wb") as file:
    pickle.dump(y_train,file)